# Session 3.2: Detector Paramaters

## Aims of this session
- Introduction to more advanced detector properties
- Learn the effect of magnification on tube-based scanners
- See how the detector's Line Spread Function dramatically change result quality (pixels are not everything!)

In [ ]:
from gvxrPython3 import gvxr, json2gvxr
import matplotlib.pyplot as plt

# Load in gvxr paramaters from a json file


# Introduction to the Detector
The Detector is a core component of all scanning systems. It is the part recieving X-rays from the beam source, converting them into signals (the images we see).

In a previous setting you quickly looked at defining a detector's spacial location, along with the number and size of pixels the detector will recieve. In this notebook, we will take a further in-depth look into how these paramaters are not enough to accurately simulate real-life detectors.

In GVXR, the detector is modelled as a simple 2D plane in the path of the beam source. This virtual detector by default has perfect properties, resulting in a theoretical perfect image. Real life detectors are unfortunately not perfect, and obtaining high quality images relies on more than resolution.

![](img/scene-detector.drawio.png)


Just like a camera, detectors can be built in many different ways. One common detector type uses scintillators to absorb x-rays and re-emit the absorbed energy as light. This light is then detected using transistors to produce an electrical image.

![](img/detector-scintillator.drawio.png)

Keep in mind that this process isn't perfect! the generated light energy is picked up by neighbouring transistors, causing blurring. Depending on the quality of the detector, this effect can drastically effect the quality of the produced scan. This phenomenon can be measured and used to calibrate detectors. The measured response is called the Line Spread Function.


## Resolution and Pixel Size

Detectors come in many different sizes, and are represented in different formats.
GVXR only supports denoting detector sizes using pixel size, and resolution.

Sometimes, the term *pixel pitch* is used. Pixel pitch is the distance between the centre of two pixels. It is often assumed that the edge of pixels have no width;- resulting in a pixel size equal to the pixel pitch.

![](img/detector-size.drawio.png)

### Task: Setting Pixel Size and Resolution

Setting the detector size in gvxr is easy, but it only supports pixel values.
The detector's resolution *must* be in pixels, you will have to convert the values yourself.

In [ ]:
# Set resolution to 512x512 with a pixel size of 200μm
gvxr.setDetectorPixelSize()

# Create a detector with a width of 8.192mm, height of 12.046mm and with a pixel size of 8μm
gvxr.setDetectorPixelSize()

# Using a resolution of 500x1000, create a detector with a width of 10mm and height of 20mm
gvxr.setDetectorPixelSize()

# Create a detector with a resolution of 512x512 with a pixel pitch of 20μm
gvxr.setDetectorPixelSize()

### Task: Pixel Stride

Pixel stride is a method to reduce noise by combining values from multiple pixels and resulting in a lower resultion image.
The result may have less total resolution, but the overall noise is decreased as more samples per pixel are used.

![](img/detector-stride.drawio.png)

GVXR does not directly support detector stride, however there are two approaches to recreate this detector feature:
1. Double the pixel size to directly reduce the resolution
2. Sample a full-resolution image

Method 2. can be done using [scikit-learn's `downscale_local_mean`](https://scikit-image.org/docs/stable/api/skimage.transform.html#skimage.transform.downscale_local_mean) but that will not be convered in this lesson.

In [12]:
# Approach 1 - Doubling pixel size

# Create a detector with a width of 10mm, height of 12mm and with a pixel size of 8μm
gvxr.setDetectorPixelSize()

refImg = gvxr.computeXRayImage()

# Create the same detector, but with a pixel size of 16μm to recreate a pixel stride of 2
gvxr.setDetectorPixelSize()

newImg = gvxr.computeXRayImage()

# -----
# Make a simple side-by-side with matplotlib
fig, axes = plt.subplots(1,2)
axes[0].imshow(refImg)
axes[1].imshow(newImg)
fig.show()

TypeError: setDetectorPixelSize() missing 3 required positional arguments: 'aWidth', 'aHeight', and 'aUnitOfLength'

Here we can see that the physical pane size has not changed, but the resolution is halved.

## OTF, MTF, PSF, ESF, and LSF

OTF, MTF, PSF, and LSF are all values used to represent the 'spillover' caused by imperfect detectors. Since scintillator detectors are not perfect there is light leakage causing other nearby pixels to be effected. This has a dramatic effect on scans.

The Optical Transfer Function (OTF) specifies how the detector system reacts to different spacial frequencies. It describes in the Fourier domain the transform of the Point Spread Function (PSF) applied to an image recieved by the detector. The Modulation Transfer Function (MTF) is defined as the magnitude of the OTF and has similar uses in different fields.

In our specific case, we are interested in the Point Spread Fuction (PSF) and Line Spread Function (LSF).
The PSF can be seen as the effect a single pixel has on the surrounding pixels, whereas LSF is a single line of this PSF though the origin.
LSF is used as the measurement as it can be calculated from an image by using a sharp gradient change, and calculating the Edge Spread Function (ESF). The first derivative of the ESF is the LSF, and the LSF is the PSF along one axis.

Confusing? Here is a useful flowchart with images to help visualise the relationships between transfer functions:

![otf.png](otf.png)

<!-- ![Detector LSF](img/detector-lsf.png) -->

Shown below are two images; one of a Siemens Star with no LSF applied, and another with a LSF calibrated from a physical detector

| Perfect Detector | Real Detector |
| --- | --- |
| ![](img/siemens-perfect.png) | ![](img/siemens-real.png) |

Notice how even with the same detector paramaters, the image is heavily blurred.
Another example with the same setup, but taken as a CT acquisition and reconstruction:

| Plate CT Perfect Detector | Plate CT Real Detector |
| --- | --- |
| ![](img/plate-recon-perfect.png) | ![](img/plate-recon-real.png) |
<small>(reconstructed using 360 projections with FDK)</small>

## Energy Response